In [134]:
# %%writefile Factor_Cacher.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import warnings
import pandas as pd
import numpy as np

import tools.Sample_Tools as smpl
import base.JuUnits as ju


__cache_dir__ = ju.get_root_path()+'/cache.feather/'
__cache_type__ = '.feather'

# df = smpl.get_sample_by_zs(name='沪深300', end='2021-11-28', gap=2500,  only_main=True, filter_st=True).data
# df.to_parquet(__cache_dir__+'test.parquet')

def file_path(file_name):
    return __cache_dir__ + file_name + __cache_type__

def is_cache_exist(cache_name):
    return os.path.exists(file_path(cache_name))

def save_cache(name, data):
    df.reset_index().to_feather(file_path(cache_name))
    
def load_cache(cache_name:str, to_series:bool=False):
    assert is_cache_exist(cache_name),'cache not exist, create first'
    df = pd.read_feather(file_path(cache_name)).set_index(['date','code'])
    if to_series:
        df =  df.squeeze()
    return df

def load_cache_adv(cache_name:str, start:str, end:str, to_series:bool=False, flag_time:str='date'):
    assert is_cache_exist(cache_name),'cache not exist, create first'
        
    st = pd.Timestamp(start) 
    en = pd.Timestamp(end) 

    df = pd.read_feather(file_path(cache_name)).set_index([flag_time,'code'])[st:en]
    date_idx = df.index.get_level_values(0).unique()
    if not (st >= date_idx.min() and en <= date_idx.max()):
        info = '"{}"：date range out of cache[{},{}]'.format(
            cache_name,
            date_idx.min().strftime('%Y-%m-%d %H:%M:%S'), 
            date_idx.max().strftime('%Y-%m-%d %H:%M:%S')
        )
        warnings.warn(info)
        
    df = df[st:en]
    
    if to_series:
        df =  df.squeeze()
    return df


def load_caches_adv(cache_names:list, start:str=None, end:str=None, flag_time:str='date'):
    if not start is None:
        assert not end is None, 'start and end must not be None at the same time,or be None at the same time'
    
    tmp = []
    for name in cache_names:
        if start is None:
            df = load_cache(name)
        else:
            df = load_cache_adv(name,start,end,flag_time=flag_time)
        tmp.append(df)  
        
    return pd.concat(tmp,axis=1)
     
    
        

In [31]:
%%time
df.reset_index().to_feather(__cache_dir__+'test.feather')

Wall time: 71.8 ms


In [137]:
# df['open'].reset_index().to_feather(__cache_dir__+'test_s.feather')
# pd.read_feather(__cache_dir__+'test_s.feather').set_index(['date','code']).squeeze()
# df.squeeze()
# load_cache_adv('test','2011-11-11','2022-11-11')
load_caches_adv(['test','test-Copy1'],'2011-11-11','2020-11-11')

open    high     low   close     volume        amount  \
date       code                                                              
2011-11-11 000001   16.77   16.84   16.51   16.57   165267.0  2.747976e+08   
           000002    7.50    7.56    7.45    7.52   270300.0  2.027468e+08   
           000063   18.35   18.40   18.07   18.16    78894.0  1.438695e+08   
           000066    6.59    6.64    6.39    6.45   341334.0  2.220610e+08   
           000069    6.89    6.99    6.76    6.80   280452.0  1.923114e+08   
...                   ...     ...     ...     ...        ...           ...   
2020-11-11 603882   93.30   93.64   87.29   87.64   117445.0  1.051053e+09   
           603899   84.86   86.44   83.68   84.00    26832.0  2.274766e+08   
           603939  104.68  108.88  103.11  103.38    37827.0  3.978887e+08   
           603986  213.00  214.88  201.12  202.01   128431.0  2.655407e+09   
           603993    4.22    4.27    4.19    4.20  1157356.0  4.895607e+08   

                     open    high     low   close     volume        amount  
date       code                                                             
2011-11-11 000001   16.77   16.84   16.51   16.57   165267.0  2.747976e+08  
           000002    7.50    7.56    7.45    7.52   270300.0  2.027468e+08  
           000063   18.35   18.40   18.07   18.16    78894.0  1.438695e+08  
           000066    6.59    6.64    6.39    6.45   341334.0  2.220610e+08  
           000069    6.89    6.99    6.76    6.80   280452.0  1.923114e+08  
...                   ...     ...     ...     ...        ...           ...  
2020-11-11 603882   93.30   93.64   87.29   87.64   117445.0  1.051053e+09  
           603899   84.86   86.44   83.68   84.00    26832.0  2.274766e+08  
           603939  104.68  108.88  103.11  103.38    37827.0  3.978887e+08  
           603986  213.00  214.88  201.12  202.01   128431.0  2.655407e+09  
           603993    4.22    4.27    4.19    4.20  1157356.0  4.895607e+08  

[449761 rows x 12 columns]